In [4]:
import gpflow
import numpy as np
import math
import tensorflow as tf
import get_history as gh
import pickle

In [2]:
sess = tf.Session()

In [57]:
 def get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two):
    #sigmax = [[] for i in range(100)]
    #tf.placeholder(tf.float64, shape=(100, 2,2))
    
            
        #print('eff')
        #print(effects.shape)
        #user_term = tf.constant(np.array([[0.0,0.0],[0.0,0.0]]))           
        


        #print('eff')
        #print(effects.shape)
        #effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
    
    
    
        
        
        get_g = g_one
        get_g_two = g_two
        
        effects_onef = [[] for i in range(100)]
        effects_twof = [[] for i in range(100)]

        
        to_return = [[] for i in range(100)]
        for i in range(100):
            temp = [[] for i in range(100)]
            tempeo = [[] for i in range(100)]
            tempet = [[] for i in range(100)]
            for j in range( 100):
                rho = rbf_custom(day_one[i], X2=day_two[j])
            #print(sess.run(rho))
                val = rho[0]
        #tf.scalar_mul(val,sigma_v)
            #print(val)
                this_sigma = tf.scalar_mul(val,sigma_v)
            #print(this_sigma)
                effects_one = tf.tensordot(get_g[i],this_sigma,axes=[[0],[1]])
                effects_two = tf.tensordot(effects_one,get_g_two[j],axes = [[0],[0]])
                
                tempeo[j]=tf.tensordot(get_g[i],sigma_v,axes=[[0],[1]])
                tempet[j]=tf.tensordot(tf.tensordot(get_g[i],sigma_v,axes=[[0],[1]]),get_g_two[j],axes = [[0],[0]])
                
                temp[j]=effects_two
                
            to_return[i]=temp
            effects_onef[i]=tempeo
            effects_twof[i]=tempet
            
            
        return to_return,effects_onef,effects_twof 

In [4]:
##need to reformat history
def put_x_together(h):
    X = []
    
    for i in range(len(h.states)):
        state_vector = h.states[i]
        state_vector.append(float(h.users[i]))
        state_vector.append(float(h.day_function[i]))
        X.append(state_vector)
    return X

In [5]:
history = gh.get_history(5,100,5)

In [6]:
X = put_x_together(history)
X=np.array(X)

In [8]:
def gather_cols(params, indices, name=None):
    """Gather columns of a 2D tensor.

    Args:
        params: A 2D tensor.
        indices: A 1D tensor. Must be one of the following types: ``int32``, ``int64``.
        name: A name for the operation (optional).

    Returns:
        A 2D Tensor. Has the same type as ``params``.
    """
    with tf.op_scope([params, indices], name, "gather_cols") as scope:
        # Check input
        params = tf.convert_to_tensor(params, name="params")
        indices = tf.convert_to_tensor(indices, name="indices")
        try:
            params.get_shape().assert_has_rank(2)
        except ValueError:
            raise ValueError('\'params\' must be 2D.')
        try:
            indices.get_shape().assert_has_rank(1)
        except ValueError:
            raise ValueError('\'params\' must be 1D.')

        # Define op
        p_shape = tf.shape(params)
        p_flat = tf.reshape(params, [-1])
        i_flat = tf.reshape(tf.reshape(tf.range(0, p_shape[0]) * p_shape[1],
                                       [-1, 1]) + indices, [-1])
        return tf.reshape(tf.gather(p_flat, i_flat),
                          [p_shape[0], -1])

In [8]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return math.exp(-((X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
rdayone = [x[6] for x in X]
rdaytwo = rdayone
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(100)] for i in range(100)])


In [20]:
    def rbf_custom( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        return tf.exp(-tf.divide(tf.square(tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
        #return tf.constant(1.0,dtype=tf.float64)

In [29]:
rbf_custom_np( 1, X2=5)

1.1253517471925912e-07

In [35]:
rbf_custom(tf.constant([1.0],dtype=tf.float64),tf.constant([5.0],dtype=tf.float64)).eval(session=sess)

array([1.12535175e-07])

In [9]:
Xt = tf.constant(X)
g_one = gather_cols(Xt, [3,4], name=None)


In [10]:
g_two=g_one


In [11]:
sigma_v =  tf.constant(np.array([[10.0,0.0],[0.0,10.0]]))


In [15]:
effects_one = tf.tensordot(tf.transpose(g_one),sigma_v,axes=[[0],[1]])
effects_two = tf.tensordot(effects_one,tf.transpose(g_two),axes = [[1],[0]])
effects_three = tf.matmul(effects_two,rhos)

In [59]:
effects_four = tf.multiply(effects_two,rhos)


In [23]:
day_one = tf.constant(rdayone,dtype=tf.float64)
day_two = tf.constant(rdaytwo,dtype=tf.float64)

In [27]:
day_two = gather_cols(Xt, [6], name=None)
day_one = gather_cols(Xt, [6], name=None)


In [58]:
test = get_all_sigmas(sigma_v,g_one,g_two,day_one,day_two)

In [41]:
ef = sess.run(test[0])


In [33]:
with open('testX.pkl','rb') as f:
    t = pickle.load(f)
    

In [35]:
users = np.array([[float(t[i][8]==t[j][8]) for j in range(100)] for i in range(100)])

In [45]:
g_one = gather_cols(t, [6,7], name=None)

In [38]:
sigma_u = tf.constant(np.array([[1.0,0.1],[0.1,1.0]]))

In [39]:
sigma_u_n = np.array([[1.0,0.1],[0.1,1.0]])

In [6]:
sigma_theta = tf.constant(np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]]))

In [21]:
sigma_theta_n = np.array([[1.1, 0.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 1.1, 0.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 1.1, 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 1.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 1.1, 0.1],
       [0.1, 0.1, 0.1, 0.1, 0.1, 1.1]])

In [160]:
def simple_check(x):
    to_return = []
    for i in range(len(x)):
        temp = []
        u_one=x[i][9]
        for j in range(len(x)):
            u_two=x[j][9]
            temp.append(rbf_custom_np(u_one,u_two))
        to_return.append(temp)
    return to_return

In [164]:
def simple_users(x):
    to_return = []
    for i in range(len(x)):
        temp = []
        u_one=x[i][8]
        for j in range(len(x)):
            u_two=x[j][8]
            temp.append(u_one==u_two)
        to_return.append(temp)
    return to_return

In [171]:
rhot = simple_check(t)
userst = simple_users(t)

In [172]:
goods = set([])
for i in range(len(rhot)):
    for j in range(len(rhot)):
        goods.add(userst[i][j]==users[i][j])

In [184]:
def one_by_one(x_one,x_two):
    b_one = x_one[:6]
    b_two = x_two[:6]
    e_one = x_one[6:8]
    e_two = x_two[6:8]
    
    u_one = x_one[8]
    u_two = x_two[8]
    
    d_one = x_one[9]
    d_two = x_two[9]
    
    bterm = np.dot(b_one.T,sigma_theta_n)
    print(bterm.shape)
    bterm = np.dot(bterm,b_two)
    print(bterm.shape)
    rho = rbf_custom_np(d_one,d_two)
    
    e_term_u_term = float(u_one==u_two)*sigma_u_n
    print(e_term_u_term.shape)
    e_term_r_term = rho*sigma_v_n
    print(e_term_r_term.shape)
    e_term = np.add(e_term_u_term,e_term_r_term)
    print(e_term.shape)
    e_term = np.dot(e_one.T,e_term)
    e_term = np.dot(e_term,e_two)
    
    return bterm+e_term

In [181]:
k = []
for xone in t:
    temp = []
    for xtwo in t:
        temp.append(one_by_one(xone,xtwo))
    k.append(temp)

In [185]:
one_by_one(t[0],t[0])

(6,)
()
(2, 2)
(2, 2)
(2, 2)


0.2886254158137378

In [180]:
def rbf_custom_np( X, X2=None):
        #print(X)
        #print(X2)
        if X2 is None:
             X2=X
        ##sigma denominator 
        return math.exp((-(X-X2)**2)/1.0)
        #tf.divide(tf.square(-tf.subtract(X,X2)),tf.constant(1.0,dtype=tf.float64)))
rdayone = [x[9] for x in t]
rdaytwo = rdayone
rhos = np.array([[rbf_custom_np( rdayone[i], X2=rdaytwo[j]) for j in range(100)] for i in range(100)])


In [151]:
tmp = np.array([np.dot(i[6:8].T,sigma_u_n) for i in t])
print(tmp.shape)
#tmp = np.dot(users,)
tmp = np.dot(users,tmp)
#tmp.shape
#tmp = np.multiply(users,tmp)


(100, 2)


In [152]:
tmp2 = np.array([np.dot(i[6:8].T,sigma_v_n) for i in t])
tmp2 = np.dot(rhos,tmp2)
tmp2.shape

(100, 2)

In [153]:
inm = np.add(tmp,tmp2)
inm.shape

(100, 2)

In [154]:
tmp3 = np.dot(inm,gs.T)
#tmp3 = np.multiply(tmp3,rhos)

In [109]:
#gs = np.array([tt[6:8] for tt in t])

In [119]:
#np.dot(tmp,gs.T)

In [120]:
tmp2 = np.array([np.dot(i[6:8].T,sigma_v_n) for i in t])
#print(tmp.shape)
tmp2 = np.dot(tmp2,gs.T)
#np.array([np.dot(tmp2[i],t[i][6:8]) for i in range(len(t))])
tmp2 = np.multiply(rhos,tmp2)

In [121]:
to_add = np.add(tmp,tmp2)

In [126]:
bs = np.array([tt[:6] for tt in t])
bases =np.array([ np.dot(t[i][:6].T,sigma_theta_n) for i in range(len(t))])
bases = np.dot(bases,bs.T)

In [155]:
f = np.add(tmp3,bases)

In [47]:
effects = tf.tensordot(tf.transpose(g_one),sigma_u,axes=[[0],[1]])

In [84]:
effects = tf.tensordot(tf.transpose(g_one),sigma_u,axes=[[0],[1]])
effects.shape
g_one.shape
effects = tf.tensordot(effects,g_one,axes = [[1],[0]])

In [91]:
sigma_v = tf.constant(np.array([[10.0,0.0],[0.0,10.0]]))
sigma_v_n = np.array([[10.0,0.0],[0.0,10.0]])

In [ ]:
effects = tf.tensordot(tf.transpose(g_one),self.sigma_u,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_one = tf.multiply(effects,self.select_users)
        

In [10]:
g_one = gather_cols(t, [0,1,2,3,4,5], name=None)


array([0.13704423, 0.07842119, 0.08709108, 0.10787279, 0.13052058,
       0.11927317])

In [31]:
np.dot(t[1][:6].T,sigma_theta_n)


array([0.16900769, 0.18051304, 0.19270994, 0.16883474, 0.19394026,
       0.15886896])

In [32]:
        effects = tf.tensordot(tf.transpose(g_one),self.sigma_u,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_one = tf.multiply(effects,self.select_users)
        
        
        
        effects = tf.tensordot(tf.transpose(g_one),self.sigma_v,axes=[[0],[1]])
       
        
        effects = tf.tensordot(effects,tf.transpose(g_two),axes = [[1],[0]])
        effects_two = tf.multiply(effects,self.rhos)
        
        effects = tf.add(effects_one,effects_two)

array([[0.20306654, 0.1444435 , 0.15311339, 0.1738951 , 0.19654289,
        0.18529548],
       [0.16900769, 0.18051304, 0.19270994, 0.16883474, 0.19394026,
        0.15886896],
       [0.17530068, 0.17210316, 0.20558964, 0.16901978, 0.1367579 ,
        0.22915186],
       [0.25127764, 0.20908923, 0.17096525, 0.23097841, 0.21108273,
        0.18065022],
       [0.22458481, 0.20656863, 0.20176628, 0.18763896, 0.22115227,
        0.18677918],
       [0.21236665, 0.19130789, 0.22363798, 0.20865415, 0.18076802,
        0.21559932],
       [0.19617418, 0.20027123, 0.19363122, 0.19776997, 0.13507307,
        0.19026146],
       [0.16415333, 0.17107733, 0.18397277, 0.22423342, 0.21688109,
        0.17442875],
       [0.18244239, 0.1604054 , 0.15096006, 0.17628733, 0.16524577,
        0.1494247 ],
       [0.19055565, 0.17687039, 0.15732275, 0.1760492 , 0.18118789,
        0.17802238],
       [0.18891691, 0.22799978, 0.23435146, 0.22593272, 0.16719313,
        0.19236233],
       [0.20106273, 0

In [60]:

eff = sess.run(effects_four)


In [61]:
np.array(eff)[0]


array([2.61161526e+000, 9.11904516e-001, 2.62467820e+000, 2.44816333e+000,
       9.79957842e-001, 2.59757154e+000, 2.56848822e+000, 4.66262582e-002,
       9.99542392e-001, 4.77759216e-002, 3.20653571e-004, 9.36039750e-001,
       4.88592941e-002, 4.71598828e-002, 3.07013339e-004, 3.38637508e-004,
       2.62522934e-007, 3.45447606e-011, 6.16160387e-016, 9.50889585e-001,
       3.11277089e-007, 3.32562799e-004, 4.54491476e-002, 2.71452458e-007,
       1.31902385e-021, 4.12423089e-028, 3.20429153e-004, 3.80123276e-011,
       1.77092716e-035, 6.08681502e-016, 1.31709134e-021, 4.19586835e-028,
       2.99624932e-007, 9.64268681e-044, 3.28498901e-011, 3.47110259e-011,
       5.94896177e-016, 6.79513350e-053, 1.28596779e-021, 6.13181664e-016,
       4.29147968e-028, 2.80012813e-007, 1.40721971e-021, 7.17450248e-063,
       4.03915698e-028, 1.77700722e-035, 9.97442177e-044, 7.22995372e-053,
       1.71482030e-035, 1.00287915e-073, 7.00314720e-063, 9.22323026e-044,
       7.83708175e-053, 1

In [186]:
M


NameError: name 'M' is not defined